In [ ]:
import psycopg2
import pandas as pd
import numpy as np
from collections import OrderedDict

First we need to connect to the database and fetch the pairs.

In [ ]:
def fetch_pairs(db_username, db_password, db_name, pair_table, word1_type, remove_infrequent = True, infrequent_count = 2):
    con = psycopg2.connect(user = username,
                       password = password,
                       host = "127.0.0.1",
                       port = "5432",
                       database = database)

    cursor = con.cursor()

    if remove_infrequent:
        cursor.execute(f"SELECT * FROM {pair_table} WHERE {word1_type} IN (SELECT {word1_type} FROM {pair_table} GROUP BY {word1_type} HAVING COUNT(*) > {infrequent_count})")
    else:
        cursor.execute(f"SELECT * FROM {pair_table}")
    
    all_pairs = cursor.fetchall() 
    
    cursor.close()
    
    return all_pairs

To connect to the database and fetch the correct table, the function requires Postgres username, password, database and table name and the type of one word in the collocation to check frequency.

In [ ]:
all_pairs = fetch_pairs(..., ..., ..., ..., ...)

The following function creates a DataFrame from the pairs. The pair parameter needs to be a list of lists or a list of tuples where each list or tuple contains both words in the collocation and the count of said collocation in the data. The count needs to be the final element. For example, it could be: pairs = [("ilus", "ilm", 256), ("ilus", "lill", 173)].

In [ ]:
def create_matrix(all_pairs, save = False, path = "df.csv"):
    dup_word1 = [pair[1] for pair in all_pairs]
    word1 = list(OrderedDict.fromkeys(dup_word1))
    
    dup_word2 = [pair[0] for pair in all_pairs]
    word2 = list(OrderedDict.fromkeys(dup_word2))
        
    data = pd.DataFrame(0, index=word1, columns=word2)
    
    for tup in all_pairs:
        data.loc[tup[1], tup[0]] = tup[2]
        
    if save:
        data.to_csv(path)
        
    return data

In [ ]:
df = create_matrix(all_pairs)